In [ ]:
from cmath import isnan
import os
from random import random
import time
from typing import Any
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from csv import writer

# read the data from a file
data = pd.read_csv(os.getcwd().replace('code/old_test_code','') + "/data/cleaned_prof_data_v4.csv")

# initialize web driver for selenium
options = webdriver.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled')
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
driver.implicitly_wait(random() * 10)


def findMaxCitations(web_elements: list):
    indOfMax = 0
    maxVal = 0
    for i in range(len(web_elements)):
        val = int(web_elements[i].text.split(" - ")[2].split(" ")[2])
        if val > maxVal:
            maxVal = val
            indOfMax = i
        else:
            continue
    return web_elements[indOfMax]


# loop through and find all professors with wrong / no google scholar ID
professors = []
professorExists = True

for i in range(13595, data.shape[0]):

    # initialize values
    scholar_id = data.iat[i, 3],
    citations = data.iat[i, 4],
    h_index = data.iat[i, 5]

    if(data.iat[i, 3] == "NOSCHOLARPAGE" or data.iat[i, 3] == "#NAME?" or isnan(data.iat[i, 4]) or isnan(data.iat[i, 5])):
        print("Name: " + data.iat[i, 0] + " | " + "University: " +
              data.iat[i, 1] + " | " + "id: " + data.iat[i, 3])

        # launch URL
        driver.get("https://scholar.google.com")

        # identify search box
        box = driver.find_element(By.NAME, "q")

        # enter search text
        box.send_keys(data.iat[i, 0] + " " + data.iat[i, 1])

        # make search
        box.send_keys(Keys.ENTER)

        # in the case that the author pops up as a recommended option
        try:
            link = driver.find_element(
                By.XPATH, "//*[@id='gs_res_ccl_mid']/div[1]/table/tbody/tr/td[2]/h4/a")
            link.click()
            time.sleep(random() * 3)
            
        except:
            try:
                # in the case there are multiple options in the recommended table
                list_of_elements = driver.find_element(
                    By.XPATH, "//*[@id='gs_res_ccl_mid']/div[1]/table/tbody/tr/td[2]").find_elements(By.TAG_NAME, "div")
                print("case 2 was executed")
                max_element = findMaxCitations(list_of_elements)
                max_element.find_element(By.TAG_NAME, "a").click()
                time.sleep(random() * 5)
            except:
                # otherwise loop through all the divs and find the correct professor
                elements_list = driver.find_element(
                    By.XPATH, "//*[@id='gs_res_ccl_mid']/div[1]").find_element(By.CLASS_NAME, "gs_a").find_elements(By.TAG_NAME, "a")
                print(elements_list)
                if elements_list: 
                    for element in elements_list:  # looping through the number of professors in the link
                        try:
                            element.find_element(By.TAG_NAME, "b")
                            element.click()
                            professorExists = True
                            break
                        except NoSuchElementException:
                            professorExists = False
                            continue  # in the case that the element is not found
                    # give time for html / css / js to load
                    time.sleep(random() * 5)
                else: 
                    professorExists = False 

        # retrieve citation, h-index, and scholar_id for that prof
        if professorExists:
            scholar_id = driver.current_url.split("=")[1].split("&")[0]
            citation = driver.find_element(
                By.XPATH, "//*[@id='gsc_rsb_st']/tbody/tr[1]/td[2]").text
            h_index = driver.find_element(
                By.XPATH, "//*[@id='gsc_rsb_st']/tbody/tr[2]/td[2]").text
            time.sleep(random() * 5)
        else:
            scholar_id = None
            citation = None
            h_index = None
        print("scholar_id: " + str(scholar_id) + " h-index: " +
              str(h_index) + " citations: " + str(citation))
        professors.append({
            "Name": data.iat[i, 0],
            "affiliation": data.iat[i, 1],
            "homepage": data.iat[i, 2],
            "scholar-id": scholar_id,
            "citations": citation,
            "h-index": h_index
        })
    else:
        continue


print(len(professors))
df = pd.DataFrame(professors)
df.to_csv("cleaned_prof_data_v5_34.csv")

[WDM] - ====== WebDriver manager ======


2022-07-20 15:47:15,416 INFO ====== WebDriver manager ======


[WDM] - Current google-chrome version is 103.0.5060


2022-07-20 15:47:15,491 INFO Current google-chrome version is 103.0.5060


[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome


...

[<selenium.webdriver.remote.webelement.WebElement (session="4ffe12982333f4536dd323f4f642bcb6", element="51b6fd74-abe5-4766-9602-1718454724bf")>]
scholar_id: None h-index: None citations: None
Name: Ron Poet | University: University of Glasgow | id: NOSCHOLARPAGE
[<selenium.webdriver.remote.webelement.WebElement (session="4ffe12982333f4536dd323f4f642bcb6", element="cc99fd7a-e2ec-401b-b3fa-933d1703c2ac")>]
scholar_id: None h-index: None citations: None
Name: John O'Donnell | University: University of Glasgow | id: NOSCHOLARPAGE
scholar_id: None h-index: None citations: None
Name: J. Paul Siebert | University: University of Glasgow | id: DoEOa80AAAAJ
scholar_id: None h-index: None citations: None
Name: Gertjan van Noord | University: University of Groningen | id: dgU9Yq0AAAAJ
scholar_id: None h-index: None citations: None
Name: Judi McCuaig | University: University of Guelph | id: NOSCHOLARPAGE
[]
scholar_id: None h-index: None citations: None
Name: Mark Wineberg | University: University 

[<selenium.webdriver.remote.webelement.WebElement (session="4ffe12982333f4536dd323f4f642bcb6", element="9edf59ff-718e-401d-a478-702cf31d3b6b")>, <selenium.webdriver.remote.webelement.WebElement (session="4ffe12982333f4536dd323f4f642bcb6", element="b7b37c8f-bb63-4da3-8dca-4581d73462f6")>]
scholar_id: None h-index: None citations: None
Name: Alexander Hinneburg | University: University of Halle-Wittenberg | id: NOSCHOLARPAGE
[<selenium.webdriver.remote.webelement.WebElement (session="4ffe12982333f4536dd323f4f642bcb6", element="79406cfd-12d7-4087-bb30-0f6fb14d6d12")>]
scholar_id: None h-index: None citations: None
Name: Matthias Müller-Hannemann | University: University of Halle-Wittenberg | id: NOSCHOLARPAGE
[<selenium.webdriver.remote.webelement.WebElement (session="4ffe12982333f4536dd323f4f642bcb6", element="696329bc-bccd-4089-a57e-6a79dc5ea5ae")>]
scholar_id: None h-index: None citations: None
Name: Klaus Reinhardt | University: University of Halle-Wittenberg | id: NOSCHOLARPAGE
[<sel

scholar_id: None h-index: None citations: None
Name: Ka-man Lam | University: University of Hong Kong | id: NOSCHOLARPAGE
[]
scholar_id: None h-index: None citations: None
Name: Jehan-Francois Paris | University: University of Houston | id: NOSCHOLARPAGE
[<selenium.webdriver.remote.webelement.WebElement (session="4ffe12982333f4536dd323f4f642bcb6", element="31694ee4-7977-414c-a726-c016a0651826")>, <selenium.webdriver.remote.webelement.WebElement (session="4ffe12982333f4536dd323f4f642bcb6", element="a739c17c-c4eb-4ee4-8ba7-007eb21b32cc")>, <selenium.webdriver.remote.webelement.WebElement (session="4ffe12982333f4536dd323f4f642bcb6", element="d3be0628-b816-4c3e-9a42-4bf5b596bc1e")>]
scholar_id: None h-index: None citations: None
Name: Lennart Johnsson | University: University of Houston | id: Sm6LugIAAAAJ
[<selenium.webdriver.remote.webelement.WebElement (session="4ffe12982333f4536dd323f4f642bcb6", element="fea3ab17-12ad-46f9-96bf-e4a710f4fcca")>, <selenium.webdriver.remote.webelement.WebE

scholar_id: JBS28ZgAAAAJ h-index: 16 citations: 1150
Name: Ammar Huneiti | University: University of Jordan | id: NOSCHOLARPAGE
scholar_id: --mc83MAAAAJ h-index: 8 citations: 276
Name: Nadim Ali Meri Obeid | University: University of Jordan | id: NOSCHOLARPAGE
[]
scholar_id: None h-index: None citations: None
Name: Reem Qadan Al Fayez | University: University of Jordan | id: NOSCHOLARPAGE
[<selenium.webdriver.remote.webelement.WebElement (session="4ffe12982333f4536dd323f4f642bcb6", element="34d51948-10fb-4b8d-a61c-b7ba578584c0")>]
scholar_id: HUFYK3EAAAAJ h-index: 3 citations: 48
Name: Moh'd B. Al-Zoubi | University: University of Jordan | id: NOSCHOLARPAGE
[<selenium.webdriver.remote.webelement.WebElement (session="4ffe12982333f4536dd323f4f642bcb6", element="1c3eacaa-4854-4d44-a907-17783ea06173")>, <selenium.webdriver.remote.webelement.WebElement (session="4ffe12982333f4536dd323f4f642bcb6", element="6aaa90ee-2156-4895-bf06-8092864200cb")>, <selenium.webdriver.remote.webelement.WebElem

[<selenium.webdriver.remote.webelement.WebElement (session="4ffe12982333f4536dd323f4f642bcb6", element="c191170c-37cb-4a10-988b-2a6b34f5680d")>, <selenium.webdriver.remote.webelement.WebElement (session="4ffe12982333f4536dd323f4f642bcb6", element="f75ebdb4-3616-4aa7-b774-49b6180ab6b6")>, <selenium.webdriver.remote.webelement.WebElement (session="4ffe12982333f4536dd323f4f642bcb6", element="5e80a1dd-027f-4208-a244-ec535adb279b")>]
scholar_id: None h-index: None citations: None
Name: Fuhua (Frank) Cheng | University: University of Kentucky | id: NOSCHOLARPAGE
[<selenium.webdriver.remote.webelement.WebElement (session="4ffe12982333f4536dd323f4f642bcb6", element="db43aee7-499c-471b-a742-0530b5ef3f48")>]
scholar_id: None h-index: None citations: None
Name: D. Manivannan | University: University of Kentucky | id: NOSCHOLARPAGE
scholar_id: None h-index: None citations: None
Name: Jerzy W. Jaromczyk | University: University of Kentucky | id: NOSCHOLARPAGE
[<selenium.webdriver.remote.webelement.

In [4]:
df

NameError: name 'df' is not defined

In [2]:
professors

NameError: name 'professors' is not defined

In [15]:
h_index

54.0

In [3]:
data

,name,affiliation,homepage,scholar_id,citations,h-index
0,P. Vassalos,AUEB,http://pubs.dbs.uni-leipzig.de/aa/aff/Greece/A...,NOSCHOLARPAGE,NaN,NaN
1,Stavros Toumpis,AUEB,https://195.251.255.130/en/faculty/toumpis-sta...,LyiVPKEAAAAJ,2804.0,21.0
2,Ion Androutsopoulos,AUEB,http://www.aueb.gr/users/ion,4UJm5EQAAAAJ,11918.0,42.0
3,Michalis K. Titsias,AUEB,http://www.aueb.gr/users/mtitsias,B-SbkAwAAAAJ,5255.0,30.0
4,Vasilios A. Siris,AUEB,http://www.aueb.gr/users/vsiris,Vj3O0cEAAAAJ,5967.0,31.0
...,...,...,...,...,...,...
20644,Ali C. Begen,Özyeğin University,https://ali.begen.net,ZIZWsXoAAAAJ,5964.0,34.0
20645,Erhan Öztop,Özyeğin University,https://robotics.ozyegin.edu.tr/members/erhan-...,A8YEQMMAAAAJ,3702.0,28.0
20646,Yasar Safkan,Özyeğin University,https://cs.ozyegin.edu.tr/en/faculty/yasarsafkan,NOSCHOLARPAGE,NaN,NaN
20647,Hasan Sözer,Özyeğin University,https://faculty.ozyegin.edu.tr/hsozer,L3FuUX0AAAAJ,847.0,16.0


In [1]:
data.loc[data['name'] == 'Claudio E. Righetti']

NameError: name 'data' is not defined

In [15]:
print(len(professors))
df = pd.DataFrame(professors)
df.to_csv("cleaned_prof_data_v5_32.csv")

16


In [ ]:
professors.append({'Name': 'Serafeim Karampogias',
  'affiliation': 'University of Athens',
  'homepage': 'None',
  'scholar-id': 'None',
  'citations': 'None',
  'h-index': 'None'})